In [3]:
!mkdir model
!tensorflowjs_converter --input_format keras "Eye Drowsiness Only Eyes RGB Extra Dataset" model/

'tensorflowjs_converter' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "tfjsmodel")

ModuleNotFoundError: No module named 'tensorflowjs'

In [1]:
import os
import numpy as np
from keras.models import load_model
import tensorflow as tf
import mediapipe as mp
import cv2
import shutil

mp_face_mesh = mp.solutions.face_mesh

In [82]:
new_data = ["Fold3_part1_26_0", "Fold3_part1_27_0", "Fold3_part1_26_5", "Fold3_part1_26_10", ""]

In [2]:
model = load_model("Eye Drowsiness Only Eyes RGB Extra Dataset")
#model1 = load_model("./Eye Drowsiness RGB")
#model2 = load_model("./Eye Drowsiness Detection")
#model3 = load_model("./Eye Drowsiness Only Eyes RGB")
#model4 = load_model("./Eye Drowsiness Only Eyes GS")
model5 = load_model("./Eye Drowsiness Only Eyes RGB 2")
model6 = load_model("./Eye Drowsiness Only Eyes RGB Cleaned")
model7 = load_model("./Eye Drowsiness Only Eyes RGB Dataset Enhanced")

In [6]:
class PredictOnlyEyesFromEyes:
    def __init__(self, path):
        self.path = path
        self.frame = None
        
    def set_frame(self, frame):
        self.frame = frame
            
    def predict_frame(self):
        frame = cv2.imread(self.path)
        frame = cv2.resize(frame, (85, 40))
        
        self.set_frame(frame)
        
        X_query = np.array([frame])
        X_query = tf.convert_to_tensor(X_query)
        pred = model6.predict([X_query])
    
        return pred
    
    def inference(self):
        pred = self.predict_frame()
                
        alert = pred[0][0]
        drowsy = pred[0][1]
        
        if (alert > drowsy):
            return ["Alert", alert, drowsy, self.frame]
        elif (alert < drowsy):
            return ["Drowsy", alert, drowsy, self.frame]
        else:
            return ["None", alert, drowsy, self.frame]

In [ ]:
"D:\Driver Drowsiness\Dataset\Fold3_part1_26_5\Frames"

In [86]:
path = "../Dataset/Fold3_part1_27_5/Frames"

for i in os.listdir(path):
    i_index = os.listdir(path).index(i)
    print(str(i_index + 1) + "/" + str(len(os.listdir(path))))
    
    predict = PredictOnlyEyes(os.path.join(path, i))
    try:
        pred = predict.inference()
    except Exception as e:
        print(e)
        continue  
        
    if pred == "Alert":
        shutil.copy(os.path.join(path, i), "../Dataset/Fold3_part1_27_5/Alert")
        continue
    if pred == "Drowsy":
        shutil.copy(os.path.join(path, i), "../Dataset/Fold3_part1_27_5/Drowsy")
        continue
  
    shutil.copy(os.path.join(path, i), "../Dataset/Fold3_part1_27_5/None")
    continue


1/626
2/626
3/626
4/626
5/626
6/626
7/626
8/626
9/626
10/626
11/626
12/626
13/626
14/626
15/626
16/626
17/626
18/626
19/626
20/626
21/626
22/626
23/626
24/626
25/626
26/626
27/626
28/626
29/626
30/626
31/626
32/626
33/626
34/626
35/626
36/626
37/626
38/626
39/626
40/626
41/626
42/626
43/626
44/626
45/626
46/626
47/626
48/626
49/626
Error Encountered!!
local variable 'mesh_points' referenced before assignment
50/626
51/626
52/626
53/626
54/626
55/626
56/626
57/626
58/626
59/626
60/626
61/626
62/626
63/626
64/626
65/626
66/626
67/626
68/626
69/626
70/626
71/626
72/626
73/626
74/626
75/626
76/626
77/626
78/626
79/626
80/626
81/626
82/626
83/626
84/626
85/626
86/626
87/626
88/626
89/626
90/626
91/626
92/626
93/626
94/626
95/626
96/626
97/626
98/626
99/626
100/626
101/626
102/626
103/626
104/626
105/626
106/626
107/626
108/626
109/626
110/626
111/626
112/626
113/626
114/626
115/626
116/626
117/626
118/626
119/626
120/626
121/626
122/626
123/626
124/626
125/626
126/626
127/626
128/626
129/62

In [4]:
class PredictOnlyEyes:
    def __init__(self, path):
        self.path = path
        self.croppedL = None
        self.croppedR = None
    
    def set_cropped(self, frameL, frameR):
        self.croppedL = frameL
        self.croppedR = frameR
    
    def mediapipe(self):
        with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.6, min_tracking_confidence=0.6) as face_mesh:
            frame = cv2.imread(self.path)
            print(frame.shape)
            results = face_mesh.process(frame)
        
            img_h, img_w = frame.shape[:2]
            
            try:
                mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
            except Exception as e:
                print("Error Encountered!!")
                
            xTL, yTL = mesh_points[27] # Top
            xLL, yLL = mesh_points[130] # Left
            xBL, yBL = mesh_points[23] # Bottom
            xRL, yRL = mesh_points[244] # Right

            xTR, yTR = mesh_points[257] # Top
            xLR, yLR = mesh_points[464] # Left
            xBR, yBR = mesh_points[253] # Bottom
            xRR, yRR = mesh_points[359] # Right
            
            rightFrame = frame[yTR - 5:yBR + 5, xLR - 5:xRR + 5]
            leftFrame = frame[yTL - 5:yBL + 5, xLL - 5:xRL + 5]

            self.set_cropped(leftFrame, rightFrame)
            
    def predict_frame(self):
        frameL = cv2.resize(self.croppedL, (85, 40))
        frameR = cv2.resize(self.croppedR, (85, 40))
        
       # frameL = cv2.cvtColor(frameL, cv2.COLOR_RGB2GRAY)
        #frameR = cv2.cvtColor(frameR, cv2.COLOR_RGB2GRAY)

        
        X_queryL = np.array([frameL])
        X_queryL = tf.convert_to_tensor(X_queryL)
        predL = model.predict([X_queryL])
        
        X_queryR = np.array([frameR])
        X_queryR = tf.convert_to_tensor(X_queryR)
        predR = model.predict([X_queryR])
        
        return [predL, predR]
    
    def inference(self):
        self.mediapipe()
        pred = self.predict_frame()
        
        predL, predR = pred       
        
#         print(predL, predR)
        
        alertL = predL[0][0]
        drowsyL = predL[0][1]
        
        alertR = predR[0][0]
        drowsyR = predR[0][1]
    
#         cv2.imshow("as", self.croppedL)
#         cv2.waitKey(0)

#         cv2.imshow("as", self.croppedR)
#         cv2.waitKey(0)

        #return pred
       
        if (alertR > drowsyR) and (alertL > drowsyL):
            return "Alert"
        if (alertR < drowsyR) and (alertL < drowsyL):
            return "Drowsy"
        
        return "None"

In [5]:
test_path = "../Dataset/Fold4_part1_38_0/Drowsy/7230.jpg"
predict = PredictOnlyEyes(test_path)
predict.inference()

(1920, 1080, 3)


'Drowsy'

In [26]:
path = "../Dataset/Fold4_part1_39_0/Frames"
dict_ = {}

for i in os.listdir(path):
    i_index = os.listdir(path).index(i)
    
    print(str(i_index) + "/" + str(len(os.listdir(path))))
    
    predict = PredictOnlyEyes(os.path.join(path, i))
    try:
        pred = predict.inference()
    except Exception as e:
        print(e)
        print("hehehe caught it!!!")
        continue
        
    if pred == "Alert":
        shutil.copy(os.path.join(path, i), "../Dataset/Fold4_part1_39_0/Alert")
    elif pred == "Drowsy":
        shutil.copy(os.path.join(path, i), "../Dataset/Fold4_part1_39_0/Drowsy")   
    else:
        continue
        #shutil.copy(os.path.join(path, i), "../Dataset/Model Only Eyes Results/No Prediction")

0/173
1/173
2/173
3/173
4/173
5/173
6/173
7/173
8/173
9/173
10/173
11/173
12/173
13/173
14/173
15/173
16/173
17/173
18/173
19/173
20/173
21/173
22/173
23/173
24/173
25/173
26/173
27/173
28/173
29/173
30/173
31/173
32/173
33/173
34/173
35/173
36/173
37/173
38/173
39/173
40/173
41/173
42/173
43/173
44/173
45/173
46/173
47/173
48/173
49/173
50/173
51/173
52/173
53/173
54/173
55/173
56/173
57/173
58/173
59/173
60/173
61/173
62/173
63/173
64/173
65/173
66/173
67/173
68/173
69/173
70/173
71/173
72/173
73/173
74/173
75/173
76/173
77/173
78/173
79/173
80/173
81/173
82/173
83/173
84/173
85/173
86/173
87/173
88/173
89/173
90/173
91/173
92/173
93/173
94/173
95/173
96/173
97/173
98/173
99/173
100/173
101/173
102/173
103/173
104/173
105/173
106/173
107/173
108/173
109/173
110/173
111/173
112/173
113/173
114/173
115/173
116/173
117/173
118/173
119/173
120/173
121/173
122/173
123/173
124/173
125/173
126/173
127/173
128/173
129/173
130/173
131/173
132/173
133/173
134/173
135/173
136/173
137/173
138/17

In [ ]:
class PredictTest:
    def __init__(self, path):
        self.path = path
        self.cropped = None
    
    def set_cropped(self, frame):
        self.cropped = frame
    
    def mediapipe(self):
        with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.6, min_tracking_confidence=0.6) as face_mesh:
            frame = cv2.imread(self.path)
            results = face_mesh.process(frame)
            
            img_h, img_w = frame.shape[:2]
            
            try:
                mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
            except Exception as e:
                print("Error Encountered!!")
                
            top = mesh_points[68][1]
            left = mesh_points[68][0]
            right = mesh_points[447][0]
            bottom = mesh_points[447][1]

            frame_cropped = frame[top:bottom, left:right]
            self.set_cropped(frame_cropped)
            
    def predict_frame(self):
        #frame = cv2.cvtColor(self.cropped, cv2.COLOR_BGR2GRAY)
        frame = cv2.imread(self.path)
        frame = cv2.resize(frame, (250, 100))
        X_query = np.array([frame])
        X_query = tf.convert_to_tensor(X_query)
        pred = model.predict([X_query])
        
        return pred
    
    def inference(self):
        #self.mediapipe()
        pred = self.predict_frame()
        
        alert = pred[0][0]
        drowsy = pred[0][1]
        
        #cv2.imshow("frame", self.cropped)
        #cv2.waitKey(0)
        
        """
        if alert > drowsy:
            print("Alert hai!")
        else:
            print("Drowsy hai!")
        """
            
        return pred

In [ ]:
class Predict:
    def __init__(self, path):
        self.path = path
        self.cropped = None
    
    def set_cropped(self, frame):
        self.cropped = frame
    
    def mediapipe(self):
        with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.6, min_tracking_confidence=0.6) as face_mesh:
            frame = cv2.imread(self.path)
            results = face_mesh.process(frame)
            
            img_h, img_w = frame.shape[:2]
            
            try:
                mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
            except Exception as e:
                print("Error Encountered!!")
                
            top = mesh_points[68][1]
            left = mesh_points[68][0]
            right = mesh_points[447][0]
            bottom = mesh_points[447][1]

            frame_cropped = frame[top:bottom, left:right]
            self.set_cropped(frame_cropped)
            
    def predict_frame(self):
        #frame = cv2.cvtColor(self.cropped, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(self.cropped, (250, 100))
        X_query = np.array([frame])
        X_query = tf.convert_to_tensor(X_query)
        pred = model.predict([X_query])
        
        return pred
    
    def inference(self):
        self.mediapipe()
        pred = self.predict_frame()
        
        alert = pred[0][0]
        drowsy = pred[0][1]
        
        cv2.imshow("frame", self.cropped)
        cv2.waitKey(0)
        
        """
        if alert > drowsy:
            print("Alert hai!")
        else:
            print("Drowsy hai!")
        """
            
        return pred

In [ ]:
"D:\Driver Drowsiness\Dataset\Model RGB 2 Results\Drowsy\4554_alert.jpg"

In [ ]:
predict = Predict("../Dataset/Model RGB 2 Results/Drowsy/3343_alert.jpg")

In [ ]:
predict.inference()

In [ ]:
predicttest = PredictTest("../Dataset/Test Model RGB 2/3343_alert.jpg")

In [ ]:
predicttest.inference()

In [ ]:
path = "../Dataset/Eye Dataset/Drowsy Raw"
dict_ = {}

for i in os.listdir(path):
    i_index = os.listdir(path).index(i)
    
    print(str(i_index) + "/" + str(len(os.listdir(path))))
    
    predict = Predict(os.path.join(path, i))iiii
    try:
        pred = predict.inference()
    except:
        print("hehehe caught it!!!")
        continue
    
    alert = pred[0][0]
    drowsy = pred[0][1]
    
    prediction = ""
    
    if alert > 0.7:
        dict_[i] = "Alert"
        shutil.copy(os.path.join(path, i), "../Dataset/Model RGB 2 Results/Alert")
        continue
    elif drowsy > 0.7:
        dict_[i] = "Drowsy"
        shutil.copy(os.path.join(path, i), "../Dataset/Model RGB 2 Results/Drowsy")
        continue
    else:
        dict_[i] = "No prediction"
        shutil.copy(os.path.join(path, i), "../Dataset/Model RGB 2 Results/No Prediction")
        continue

In [ ]:
alert = 0
drowsy = 0
np_ = 0

for i in dict_.keys():
    if dict_[i] == "Alert":
        alert+= 1
    if dict_[i] == "Drowsy":
        drowsy +=1
    if dict_[i] == "No prediction":
        np_ += 1

In [ ]:
drowsy, alert, np_

In [ ]:
dict_